
# Conditional autoregressive transformer

Train a transformer to predict missing pixel from mnist 

### plan

* note to try padded mnist (relative encoding might require black padding???)
* probably don't need positional encoding?
* create transformer model
* masking 
* randomised masking
* relative position encoding (x - current_x, y - current_y, val)
* train to predict when current pixel missing
* train to predict when 10% are missing
* train to predict when 90% are missing
* train to predict when 99% are missing



In [1]:
model_name = "txformer-bigger-3"

In [2]:
# Init weights and biases project

import wandb
from wandb.keras import WandbCallback


wandb.init(project='conditional-mnist', entity='maxeonyx')
config = wandb.config
config.learning_rate = 0.01

wandb: Currently logged in as: maxeonyx (use `wandb login --relogin` to force relogin)


In [3]:
# reserve GPU 0 only (for VUW machines)

import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
# start tensorflow

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

tf.constant([1])

2021-09-22 15:01:28.569562: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-22 15:01:29.132983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6668 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:3b:00.0, compute capability: 7.5


<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [5]:
from PIL import Image, ImageDraw
from IPython.display import display

def display_uint8_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display(Image.fromarray(image, "L"))

def display_float32_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display_uint8_image(image.astype(np.uint8))

In [6]:
def idxs_to_onehots(idxs, depth=784):
    onehots = tf.one_hot(idxs, depth, dtype=tf.bool, on_value=False, off_value=True)
    return onehots

# takes 2D tensor (batch and index list)
def idxs_to_multihot(idxs, depth=784):
    onehots = idxs_to_onehots(idxs, depth)
    multihot = tf.math.reduce_all(onehots, axis=len(onehots.shape)-2)
    return multihot

def idxs_to_attention_mask(idxs):
    multihot = idxs_to_multihot(idxs)
    attn_mask = tf.logical_and(multihot[:, :, None], multihot[:, None, :])
    return attn_mask

def mask_to_image_mask(masks):
    image_masks = tf.reshape(masks, [-1, 28, 28])
    return image_masks

def img_to_tuples(img):
    height, width, chan = img.shape
    vals = tf.reshape(img, [width*height, chan])
    vals = tf.cast(vals, tf.float32)
    rows = tf.range(height, dtype=tf.float32)
    cols = tf.range(width, dtype=tf.float32)
    cols, rows = tf.meshgrid(rows, cols)
    rows = tf.reshape(rows, [-1, 1])
    cols = tf.reshape(cols, [-1, 1])
    tups = tf.concat([vals, rows, cols], axis=-1)
    return tups

def random_mask():
    idxs = tf.range(784)
    idxs = tf.random.shuffle(idxs)
    n = tf.random.uniform(shape=[], maxval=784, dtype=tf.int32)
    idxs = idxs[:n]
    return idxs_to_multihot(idxs)

def random_square_mask(maxsize=28):
    height = tf.random.uniform(shape=[], minval=1, maxval=maxsize, dtype=tf.int32)
    width = tf.random.uniform(shape=[], minval=1, maxval=maxsize, dtype=tf.int32)
    start_row = tf.random.uniform(shape=[], minval=0, maxval=maxsize-height, dtype=tf.int32)
    start_col = tf.random.uniform(shape=[], minval=0, maxval=maxsize-width, dtype=tf.int32)
    rows = tf.range(start_row, start_row + height)
    cols = tf.range(start_col, start_col + width)
    cols, rows = tf.meshgrid(rows, cols)
    idxs = rows*maxsize+cols
    print(idxs)
    idxs = tf.reshape(idxs, [-1])
    return idxs_to_multihot(idxs, depth=maxsize*maxsize)

def random_offset():
    return tf.random.uniform(shape=[2], maxval=28, dtype=tf.int32)
    
def display_mask(mask):
    image_mask = np.array(mask_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    image_mask = image_mask * 255
    display_uint8_image(image_mask)

In [7]:
tf.reshape(random_square_mask(5), [5,5])

tf.Tensor(
[[11 12]
 [16 17]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(5, 5), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True, False, False,  True,  True],
       [ True, False, False,  True,  True],
       [ True,  True,  True,  True,  True]])>

In [8]:
# tensorflow.data data generator

from tensorflow import data as td
import tensorflow_datasets as tfds

def make_dataset_generator(split='train', batch_size=None):
    
    dataset = tfds.load('mnist', split=split, shuffle_files=True)
    
    print("Start:", dataset)
    
    # shuffle the digits
    dataset = dataset.shuffle(60000)
    # repeat the dataset infinitely
    dataset = dataset.repeat()
    
    def map_just(component, func):
        def do(data):
            data[component] = func(data[component])
            return data
        return do
    
    def map_add(component, func):
        def do(data):
            data[component] = func()
            return data
        return do
    
    def add_tuples(data):
        data['tuples'] = img_to_tuples(data['image'])
        return data
    
    def offset_tuples(data):
        offset = tf.cast(data['offset'], tf.float32)
        data['tuples'] = tf.stack([
            data['tuples'][:, 0],
            data['tuples'][:, 1] + offset[0],
            data['tuples'][:, 1] + offset[1],
        ], axis=-1)
        return data
    
    def add_target_px_val(data):
        offset = data['offset']
        data['target_px'] = tf.cast(data['image'][offset[0],offset[1]], tf.float32)
        return data
    
    def add_square_mask(data):
        mask = data['mask']
        square_mask = random_square_mask
        mask = tf.logical_and(mask, square_mask)
        data['mask'] = mask
        return data
    
    dataset = dataset.map(add_tuples)
    # data['image'][:, 0] are vals
    # data['image'][:, 1] are rows
    # data['image'][:, 2] are cols
    
    dataset = dataset.map(map_add('mask', random_mask))
    dataset = dataset.map(map_add('offset', random_offset))
    dataset = dataset.map(offset_tuples)
    dataset = dataset.map(add_target_px_val)
    
    if batch_size is not None:
        dataset = dataset.batch(batch_size, drop_remainder=True)
    
    return dataset

make_dataset_generator(batch_size=8)

Start: <_OptionsDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>


<BatchDataset shapes: {image: (8, 28, 28, 1), label: (8,), tuples: (8, 784, 3), mask: (8, 784), offset: (8, 2), target_px: (8, 1)}, types: {image: tf.uint8, label: tf.int64, tuples: tf.float32, mask: tf.bool, offset: tf.int32, target_px: tf.float32}>

In [9]:
# reformat MNIST data into (x, y, val) tuples

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)



# [row, col, value] tuples
def mnist_to_rcv_tuples(dataset):
    # positional encoding is just indices as floats
    x_idx = np.arange(dataset.shape[1]).astype(np.float32)
    y_idx = np.arange(dataset.shape[2]).astype(np.float32)

    # make 28x28 cartesian product and repeat N times
    idxs = np.transpose([np.repeat(y_idx, len(x_idx)),np.tile(x_idx, len(y_idx))])
    
    all_idxs = np.tile(idxs.reshape(-1), dataset.shape[0]).reshape(-1,784,2)
    # pixel data as float between 0 and 255
    all = dataset.astype(np.float32).reshape(-1, 784, 1)
    
    big = np.dstack((all_idxs, all))
    
    return big

def random_offset_tuples(dataset, dataset_y, min_masked_px=0, max_masked_px=784, random_squares=True):
    tuples = mnist_to_rcv_tuples(dataset)
    offsets = np.random.randint(0, 28, [dataset.shape[0], 2])
    mask_sizes = np.random.randint(min_masked_px, max_masked_px, [dataset.shape[0]])
    tuples[:, :, 0] -= offsets[:, 0, np.newaxis] # row (y) offset
    tuples[:, :, 1] -= offsets[:, 1, np.newaxis] # col (x) offset
    target_idxs = offsets[:, 0] * 28 + offsets[:, 1]
    
    # permutations 
    masks = np.ones([dataset.shape[0], 784], dtype=np.bool_)
    for i in range(dataset.shape[0]):
        # shuffle tuples
        tup_shuffle_idxs = np.random.permutation(784)
        tuples[i] = tuples[i, tup_shuffle_idxs]
        
        idxs = np.random.permutation(784)
                           
        n_mask_px = np.random.randint(min_masked_px, max_masked_px)
        # n mask plus always mask the target idx
        mask_idxs = np.concatenate([idxs[:n_mask_px], target_idxs[i:i+1]])
        
        if random_squares:
            # random square cut out of the masks
            start_row, start_col = np.random.randint(0, 28, [2])
            end_row = np.random.randint(start_row, 28)
            end_col = np.random.randint(start_col, 28)
            height, width = abs(end_row - start_row), abs(end_col - start_col)
            square_row_idxs, square_col_idxs = np.indices([height, width])
            square_row_idxs += start_row
            square_col_idxs += start_col
            square_mask_idxs = square_row_idxs * 28 + square_col_idxs
            square_mask_idxs = square_mask_idxs.reshape(-1)
            
            mask_idxs = np.concatenate([mask_idxs, square_mask_idxs])
            
        masks[i:i+1] = idxs_to_multihot(mask_idxs)
    
    
    
    target_pxs = dataset[np.arange(dataset.shape[0]), offsets[:, 0], offsets[:, 1]].astype(np.float32)
    return {
        "digits": tf.cast(dataset_y, tf.uint8),
        "imgs": tf.cast(dataset, tf.uint8),
        "tuples": tf.cast(tuples, tf.float32),
        "offsets": offsets,
        "target_pxs": target_pxs,
        "target_idxs": target_idxs,
        "masks": tf.cast(masks, tf.bool)
    }

def load_to_tensorflow(file):
    dataset = np.load(file)
    return { k: tf.constant(v) for k, v in dataset.items() }
    
load_saved_dataset = True
if not load_saved_dataset:
    train_dataset = random_offset_tuples(x_train, y_train)
    test_dataset = random_offset_tuples(x_test, y_test)
    # give very few pixels
    full_masks_dataset = random_offset_tuples(x_test, y_test, min_masked_px=770, max_masked_px=784, random_squares=True)
    np.savez("train_dataset.npz", **train_dataset)
    np.savez("test_dataset.npz", **test_dataset)
    np.savez("full_masks_dataset.npz", **full_masks_dataset)
else:
    train_dataset = load_to_tensorflow("train_dataset.npz")
    test_dataset = load_to_tensorflow("test_dataset.npz")
    full_masks_dataset = load_to_tensorflow("full_masks_dataset.npz")
    

In [10]:
def dataset_summary(d):
    for name, v in d.items():
        print(name, "shape", v.shape)

    for i in range(4):
        print("index", i, "which is a", d["digits"][i])
        display_uint8_image(d["imgs"][i])
        display_mask(d["masks"][i])
        display_uint8_image(tf.constant(d["imgs"][i]) * tf.cast(mask_to_image_mask(d["masks"][np.newaxis, i])[0], tf.uint8))

dataset_summary(train_dataset)    
dataset_summary(test_dataset)
dataset_summary(full_masks_dataset)


digits shape (60000,)
imgs shape (60000, 28, 28)
tuples shape (60000, 784, 3)
offsets shape (60000, 2)
target_pxs shape (60000,)
target_idxs shape (60000,)
masks shape (60000, 784)
index 0 which is a tf.Tensor(5, shape=(), dtype=uint8)


index 1 which is a tf.Tensor(0, shape=(), dtype=uint8)


index 2 which is a tf.Tensor(4, shape=(), dtype=uint8)


index 3 which is a tf.Tensor(1, shape=(), dtype=uint8)


digits shape (10000,)
imgs shape (10000, 28, 28)
tuples shape (10000, 784, 3)
offsets shape (10000, 2)
target_pxs shape (10000,)
target_idxs shape (10000,)
masks shape (10000, 784)
index 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


...

index 3 which is a tf.Tensor(0, shape=(), dtype=uint8)


digits shape (10000,)
imgs shape (10000, 28, 28)
tuples shape (10000, 784, 3)
offsets shape (10000, 2)
target_pxs shape (10000,)
target_idxs shape (10000,)
masks shape (10000, 784)
index 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


index 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


index 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


index 3 which is a tf.Tensor(0, shape=(), dtype=uint8)


In [11]:

# f_masks = np.array([29, 31, 754, 752])
# print(f_masks)
# f_masks = idxs_to_multihot(f_masks)
# print(f_masks)
# display_mask(f_masks)

# t_masks = np.indices([784])
# print(t_masks)
# t_masks = idxs_to_multihot(t_masks)
# print(t_masks)
# display_mask(t_masks)

In [12]:

print(test_dataset["tuples"].shape, test_dataset["offsets"].shape, test_dataset["target_pxs"].shape)
print(test_dataset["tuples"][0])
print(test_dataset["offsets"][0])
print(test_dataset["target_pxs"][0])

# actual training dataset is 784 x,y,v tuples, but with the spatial location changed so the the pixel to predict is in the center
# and the pixel value to predict removed (via masking)

# then, can mask more and more stuff and compare performance

# we can also use the model in an "iterative" mode to generate images, in an arbitrary order.

(10000, 784, 3) (10000, 2) (10000,)
tf.Tensor(
[[ 18. -15.   0.]
 [  0. -20.   0.]
 [ 13.  -7. 240.]
 ...
 [  6.  -5.  83.]
 [ -3.  -9.   0.]
 [  2. -22.   0.]], shape=(784, 3), dtype=float32)
tf.Tensor([ 5 23], shape=(2,), dtype=int64)
tf.Tensor(0.0, shape=(), dtype=float32)


In [13]:
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
# attention manual calculation example

m = tf.constant([[5, 3], [1, 2]], dtype=tf.float32)
val = tf.constant([[5, 3], [1, 4], [1, 6]], dtype=tf.float32)
print(val)
mask = tf.constant([1, 0, 1], dtype=tf.float32)
wei = tf.constant([[1, 2, 3], [0.5, 2, 2], [1, 3, 1]], dtype=tf.float32)
wei = keras.layers.Softmax()(wei, mask)
print(wei)
val = tf.expand_dims(val, -3)
print(val.shape)
wei = tf.expand_dims(wei, -1)
print(wei.shape)
x = val * wei
print(x.shape)
print(x)
mask = tf.expand_dims(mask, -1)
# x = x * mask
print(x)
x = tf.reduce_sum(x, axis=-2)
print(x)

tf.Tensor(
[[5. 3.]
 [1. 4.]
 [1. 6.]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.11920292 0.         0.880797  ]
 [0.18242551 0.         0.81757444]
 [0.5        0.         0.5       ]], shape=(3, 3), dtype=float32)
(1, 3, 2)
(3, 3, 1)
(3, 3, 2)
tf.Tensor(
[[[0.5960146  0.35760877]
  [0.         0.        ]
  [0.880797   5.2847824 ]]

 [[0.91212755 0.54727656]
  [0.         0.        ]
  [0.81757444 4.9054465 ]]

 [[2.5        1.5       ]
  [0.         0.        ]
  [0.5        3.        ]]], shape=(3, 3, 2), dtype=float32)
tf.Tensor(
[[[0.5960146  0.35760877]
  [0.         0.        ]
  [0.880797   5.2847824 ]]

 [[0.91212755 0.54727656]
  [0.         0.        ]
  [0.81757444 4.9054465 ]]

 [[2.5        1.5       ]
  [0.         0.        ]
  [0.5        3.        ]]], shape=(3, 3, 2), dtype=float32)
tf.Tensor(
[[1.4768116 5.642391 ]
 [1.729702  5.452723 ]
 [3.        4.5      ]], shape=(3, 2), dtype=float32)


## Attention Maths

Dimensions $N$, $D$, $E$ and $B$.

- $N = 784$ is the number of inputs.
- $D$ is the width of the _key_ $K$ and _query_ $Q$ vectors.
- $E$ is the width of the _value_ vectors $V$.
- There is also a (or multiple) batch dimension(s) $B$.

$K$ is $B \times N \times D$ dimensional.
$Q$ is $B \times N \times D$ dimensional.
$V$ is $B \times N \times E$ dimensional.
Because it is self-attention, $K$ and $Q$ have the same length $N$, and the attention matrix is square.
The attention matrix is $A = Q \cdot K^T$, and is $B \times N \times N$ dimensional. Formally:
$$
A_{b,i,j} = \sum_d Q_{b,i,d} K_{b,j,d}
$$

We do softmax normalization along the columns $j$ of the attention matrix (such that each _row_ $i$ sums to 1). The result is the attention weights. Formally:
$$
\bar{A}_{b,i,j} = \frac{e^{A_{b,i,j}}}{\sum_{j'} e^{A_{b,i,j'}}}
$$

The output $O$ of the attention layer is $B \times N \times E$ dimensional. It is obtained by the attention weights multiplied by the value vectors $V$. $A$ is $B \times N \times N$ dimensional and $V$ is $B \times N \times E$ dimensional.
$$
    O_{b,i,e} = \sum_j A_{b,i,j} V_{b,j,e}
$$

Often the dimensions $E = D$ because this allows multiple attention layers in sequence, but this need not be the case.

In [15]:



def multi_head_attention(n_heads, n_kq_dim, n_val_dim):
    
    k_dense = layers.Dense(n_kq_dim, activation='linear')
    q_dense = layers.Dense(n_kq_dim, activation='linear')
    
    
    
    softmax = layers.Softmax(axis=-1)
    
    val_dense = layers.Dense(n_val_dim, activation='relu')
    
    def call(inputs, mask):
        
        k = k_dense(inputs)
        q = q_dense(inputs)
        
        scores = tf.matmul(query, key, transpose_b=True)
        weights = softmax(scores, mask)
        
        vals = val_dense(inputs)
        
        vals = tf.expand_dims(-1)
        weights = tf.expand_dims(-2)
        
        outputs = tf.reduce_sum(vals * weights)
        
        
        vals *= mask
        

def transformer_block(n_embed_dim, n_heads, n_dense_dim, dropout_rate):
    attn = layers.MultiHeadAttention(num_heads=n_heads, key_dim=n_embed_dim)
    dense_net_1 = layers.Dense(n_dense_dim, activation='relu')
    dense_net_2 = layers.Dense(n_embed_dim)
    layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    dropout1 = layers.Dropout(dropout_rate)
    dropout2 = layers.Dropout(dropout_rate)
    
    def call(inputs, masks, include_residual):
        mask = tf.logical_and(masks[:, :, None], masks[:, None, :])
        attn_output = attn(inputs, inputs, attention_mask=mask)
        attn_output = dropout1(attn_output)
        if include_residual:
            attn_output = inputs + attn_output
        # mask outputs. important! without, model learns magic powers (can detect and use verrrrrrry small numbers which are not literally 0)
        attn_output = attn_output * tf.expand_dims(tf.cast(masks, tf.float32), -1)
        attn_output = layernorm1(attn_output)
        dense_output = dense_net_1(attn_output)
        dense_output = dense_net_2(dense_output)
        dense_output = dropout2(dense_output)
        return layernorm2(attn_output + dense_output)
    
    return call
    



In [16]:
def model(batch_size):

    # no batch size to start makes it simpler
    n_embd = 20

    data_input = keras.Input(shape=[784, 3], batch_size=None)
    mask_input = keras.Input(shape=[784], batch_size=None, dtype=tf.bool)
    m = data_input

    # make n_embd-dimensional input embeddings per pixel from [x, y, v]
    m = layers.Dense(200, activation='relu')(m)
    m = layers.Dense(n_embd, activation='relu')(m)
    m = transformer_block(n_embed_dim=n_embd, n_heads=12, n_dense_dim=200, dropout_rate=0.1)(m, masks=mask_input, include_residual=False)
    m = transformer_block(n_embed_dim=n_embd, n_heads=12, n_dense_dim=200, dropout_rate=0.1)(m, masks=mask_input, include_residual=True)
    m = transformer_block(n_embed_dim=n_embd, n_heads=12, n_dense_dim=200, dropout_rate=0.1)(m, masks=mask_input, include_residual=True)
    m = transformer_block(n_embed_dim=n_embd, n_heads=12, n_dense_dim=200, dropout_rate=0.1)(m, masks=mask_input, include_residual=True)
    m = layers.Flatten()(m)
    m = layers.Dense(10, activation='relu')(m)
    m = layers.Dense(1, activation=None)(m)

    model = keras.Model(inputs=[data_input, mask_input], outputs=[m])

    return model


In [17]:
batch_size=16

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
lr_metric = get_lr_metric(optimizer)

txformer = model(batch_size)
txformer.compile(optimizer=optimizer, loss='mse', metrics=['MeanAbsoluteError', lr_metric])

def fit_one_epoch(dataset):
    txformer.fit(x=[dataset["tuples"], dataset["masks"]], y=dataset["target_pxs"], epochs=1, batch_size=batch_size, callbacks=[WandbCallback()])

load_saved_model = False
if load_saved_model:
    txformer.load_weights(f"./models/{model_name}")

txformer.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784, 3)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 784, 200)     800         input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 784, 1)       0           input_2[0][0]                    
______________________________________________________________________________________________

In [18]:


# display:
# - before mask
# - mask
# - after mask
# - prediction
def gen_image(dataset, idx):
    img = np.zeros([28, 28])
    mask = dataset["masks"][idx]
    erow, ecol = dataset["offsets"][idx]

    image_mask = np.array(mask_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    masked_image = np.copy(dataset["imgs"][idx])
    
    print("MNIST idx", idx, "which is a", dataset['digits'][idx])
    
    display_uint8_image(masked_image) # before mask
    display_uint8_image(image_mask * 255) # mask
    masked_image = masked_image * image_mask
    display_uint8_image(masked_image) # after mask
    
    for row in range(28):
        for col in range(28):
            img_tups = np.copy(dataset["tuples"][idx])
            img_tups[:, 0] += erow
            img_tups[:, 1] += ecol
            img_tups[:, 0] -= row
            img_tups[:, 1] -= col
            mask_idx = row*28+col
            targ_idx_mask = idxs_to_onehots(tf.constant([mask_idx]))
            this_mask = tf.logical_and(mask, targ_idx_mask)
            px = txformer([img_tups[np.newaxis, :, :], this_mask])
            img[row, col] = np.clip(px[0, 0], 0, 255)
            
    display_float32_image(img)

def image_performance_test():
    gen_image(test_dataset, 0)
    gen_image(test_dataset, 1)
    gen_image(test_dataset, 2)
    
    # full masks dataset as sanity test
    gen_image(full_masks_dataset, 0)
    gen_image(full_masks_dataset, 1)
    gen_image(full_masks_dataset, 2)


In [19]:
#image_performance_test()

In [20]:
fit_one_epoch(train_dataset)

2021-09-22 15:01:34.122159: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


   6/3750 [..............................] - ETA: 9:34 - loss: 8805.0303 - mean_absolute_error: 45.7802 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0141s vs `on_train_batch_end` time: 0.1296s). Check your callbacks.


3750/3750 [==============================] - 565s 150ms/step - loss: 4876.7896 - mean_absolute_error: 41.4980 - lr: 9.9997e-05


In [21]:
image_performance_test()


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


In [22]:
fit_one_epoch(train_dataset)

   6/3750 [..............................] - ETA: 9:37 - loss: 5119.9893 - mean_absolute_error: 41.7704 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0185s vs `on_train_batch_end` time: 0.1372s). Check your callbacks.


3750/3750 [==============================] - 565s 151ms/step - loss: 4497.1680 - mean_absolute_error: 38.3516 - lr: 9.9997e-05


In [23]:
image_performance_test()


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


In [24]:
fit_one_epoch(train_dataset)

   6/3750 [..............................] - ETA: 9:41 - loss: 3136.9148 - mean_absolute_error: 29.6187 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0196s vs `on_train_batch_end` time: 0.1369s). Check your callbacks.


3750/3750 [==============================] - 568s 152ms/step - loss: 4310.6152 - mean_absolute_error: 37.1837 - lr: 9.9997e-05


In [25]:
image_performance_test()


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


In [26]:


if not load_saved_model:
    txformer.save_weights(f"./models/{model_name}")

In [27]:
for i in range(20):
    gen_image(test_dataset, i)

MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


MNIST idx 3 which is a tf.Tensor(0, shape=(), dtype=uint8)


MNIST idx 4 which is a tf.Tensor(4, shape=(), dtype=uint8)


...

MNIST idx 15 which is a tf.Tensor(5, shape=(), dtype=uint8)


MNIST idx 16 which is a tf.Tensor(9, shape=(), dtype=uint8)


MNIST idx 17 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 18 which is a tf.Tensor(3, shape=(), dtype=uint8)


MNIST idx 19 which is a tf.Tensor(4, shape=(), dtype=uint8)


In [28]:
fit_one_epoch(train_dataset)
fit_one_epoch(train_dataset)
fit_one_epoch(train_dataset)
fit_one_epoch(train_dataset)

   6/3750 [..............................] - ETA: 9:44 - loss: 4306.9199 - mean_absolute_error: 34.2244 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0166s vs `on_train_batch_end` time: 0.1397s). Check your callbacks.


   6/3750 [..............................] - ETA: 9:48 - loss: 1972.4360 - mean_absolute_error: 20.9936 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0175s vs `on_train_batch_end` time: 0.1405s). Check your callbacks.


   6/3750 [..............................] - ETA: 9:38 - loss: 611.5412 - mean_absolute_error: 12.0403 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0182s vs `on_train_batch_end` time: 0.1375s). Check your callbacks.


   6/3750 [..............................] - ETA: 9:41 - loss: 2359.3250 - mean_absolute_error: 22.4406 - lr: 1.0000e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0161s vs `on_train_batch_end` time: 0.1398s). Check your callbacks.


3750/3750 [==============================] - 569s 152ms/step - loss: 1891.8329 - mean_absolute_error: 19.2316 - lr: 9.9997e-05


In [ ]:
for i in range(20):
    gen_image(test_dataset, i)

MNIST idx 0 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 1 which is a tf.Tensor(2, shape=(), dtype=uint8)


MNIST idx 2 which is a tf.Tensor(1, shape=(), dtype=uint8)


MNIST idx 3 which is a tf.Tensor(0, shape=(), dtype=uint8)


MNIST idx 4 which is a tf.Tensor(4, shape=(), dtype=uint8)


...

MNIST idx 15 which is a tf.Tensor(5, shape=(), dtype=uint8)


MNIST idx 16 which is a tf.Tensor(9, shape=(), dtype=uint8)


MNIST idx 17 which is a tf.Tensor(7, shape=(), dtype=uint8)


MNIST idx 18 which is a tf.Tensor(3, shape=(), dtype=uint8)


MNIST idx 19 which is a tf.Tensor(4, shape=(), dtype=uint8)


In [ ]:


if not load_saved_model:
    txformer.save_weights(f"./models/{model_name}")